## First make folder in colab 

1.   small_data
2.   complete_data

## Upload data 25000 pictures in complete_data folder from your PC.

## Connect Drive with Colab
There is uploaded folder train 
containing 25000 pictures directory:
complete_data_dir="/content/drive/My Drive/train"

##Then make folder in drive small_data


In [1]:
import os
from google.colab import drive
#drive.flush_and_unmount()
drive.mount('/content/drive/',force_remount=True)

Mounted at /content/drive/


In [2]:
small_data_dir = "/content/drive/My Drive/small_data"
complete_data_dir="/content/drive/My Drive/train"

In [3]:
train_dir = os.path.join(small_data_dir,'train_data')
os.mkdir(train_dir)

FileExistsError: ignored

In [4]:
valid_dir = os.path.join(small_data_dir,'valid_data')
os.mkdir(valid_dir)

FileExistsError: ignored

In [5]:
test_dir = os.path.join(small_data_dir,'test_data')
os.mkdir(test_dir)

FileExistsError: ignored

In [6]:
train_cat_dir = os.path.join(train_dir,'train_cats')
os.mkdir(train_cat_dir)

FileExistsError: ignored

In [7]:
#@title Default title text
train_dog_dir = os.path.join(train_dir,"train_dogs")
os.mkdir(train_dog_dir)#

FileExistsError: ignored

In [8]:
valid_cat_dir = os.path.join(valid_dir,"valid_cats")
os.mkdir(valid_cat_dir)

FileExistsError: ignored

In [9]:
valid_dog_dir = os.path.join(valid_dir,"valid_dogs")
os.mkdir(valid_dog_dir)

FileExistsError: ignored

In [10]:
test_cat_dir = os.path.join(test_dir,"test_cats")
os.mkdir(test_cat_dir)


FileExistsError: ignored

In [11]:
test_dog_dir = os.path.join(test_dir,"test_dogs")
os.mkdir(test_dog_dir)

FileExistsError: ignored

# Image Preprocessing
## Data Augmentation

### Read Data from Directories using Generator Image Data

In [12]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
rescale=1./255,
rotation_range=40,
width_shift_range=0.2,
height_shift_range=0.2,
shear_range=0.2,
zoom_range=0.2,
horizontal_flip=True,
fill_mode='nearest')

In [13]:
train_generator = train_datagen.flow_from_directory(
                                            train_dir,
                                            target_size=(150, 150),
                                            batch_size=20,
                                            class_mode='binary')

Found 2000 images belonging to 2 classes.


In [14]:
valid_datagen = ImageDataGenerator(rescale=1./255)

In [15]:
valid_generator = valid_datagen.flow_from_directory(
                                            valid_dir,
                                            target_size=(150, 150),
                                            batch_size=20,
                                            class_mode='binary')

Found 1000 images belonging to 2 classes.


In [16]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [17]:
for data_batch, labels_batch in train_generator:
  
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

data batch shape: (20, 150, 150, 3)
labels batch shape: (20,)


In [18]:
for data_batch, labels_batch in valid_generator:
  
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

data batch shape: (20, 150, 150, 3)
labels batch shape: (20,)


# Build Pretained Model

In [ ]:
# Model 

In [ ]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [19]:
from tensorflow.keras.applications import VGG16
conv_base = VGG16(weights='imagenet',
include_top=False,
input_shape=(150, 150, 3))

58900480/58889256 [==============================] - 1s 0us/step


In [20]:
from keras import models
from keras import layers
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(130, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [22]:
print('This is the number of trainable weights ''before freezing the conv base:', len(model.trainable_weights))
conv_base.trainable = False
print('This is the number of trainable weights ''after freezing the conv base:', len(model.trainable_weights))


This is the number of trainable weights before freezing the conv base: 4
This is the number of trainable weights after freezing the conv base: 4


In [ ]:
# Compile

In [23]:
from tensorflow.keras import optimizers

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['acc'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


## Fitting Model

In [26]:
history = model.fit_generator(
train_generator,
steps_per_epoch=100,
epochs=30,
validation_data=valid_generator,
validation_steps=50)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
100/100 [==============================] - 29s 285ms/step - loss: 0.2711 - acc: 0.8785 - val_loss: 0.2450 - val_acc: 0.9050
Epoch 2/30
100/100 [==============================] - 28s 282ms/step - loss: 0.2875 - acc: 0.8780 - val_loss: 0.2449 - val_acc: 0.9040
Epoch 3/30
100/100 [==============================] - 29s 285ms/step - loss: 0.2717 - acc: 0.8845 - val_loss: 0.2498 - val_acc: 0.8990
Epoch 4/30
100/100 [==============================] - 28s 283ms/step - loss: 0.2711 - acc: 0.8770 - val_loss: 0.2491 - val_acc: 0.8950
Epoch 5/30
100/100 [==============================] - 28s 284ms/step - loss: 0.2790 - acc: 0.8840 - val_loss: 0.2409 - val_acc: 0.9030
Epoch 6/30
100/100 [==============================] - 28s 284ms/step - loss: 0.2709 - acc: 0.8855 - val_loss: 0.2437 - val_acc: 0.9020
Epoch 7/30
100/100 [==============================] - 29s 288ms/step - loss: 0.2763 - acc: 0.8800 - val_loss: 0.2468 - val_acc: 0.9000
Epoch 8/30
100/100 [==============================] - 2

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

NameError: ignored